<a href="https://colab.research.google.com/github/dharvi-t/employee-attrition-prediction-HR-analytics-/blob/main/employee_attrition_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 HR Analytics & Employee Attrition Prediction
**Author:** Dharvi Tomar   
**Objective:** Predict employee attrition and provide insights using EDA, SQL, and ML.  
**Dataset:** IBM HR Analytics Attrition Dataset  


## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sqlite3
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

# 2. Load Data

In [ ]:
url = "https://raw.githubusercontent.com/dharvi-t/employee-attrition-prediction-HR-analytics-/main/WA_Fn-UseC_-HR-Employee-Attrition.csv"
df = pd.read_csv(url)
df.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


# 2. Database Connection

In [ ]:
# Create a file-based SQLite database
conn = sqlite3.connect("employee_data.db")

# Write pandas DataFrame to SQL
df.to_sql("employee_attrition", conn, index=False, if_exists="replace")

# Run SQL queries
query = "SELECT Department, AVG(MonthlyIncome) as AvgIncome FROM employee_attrition GROUP BY Department"
result = pd.read_sql(query, conn)
print(result.head())

# Close the connection
conn.close()

               Department    AvgIncome
0         Human Resources  6654.507937
1  Research & Development  6281.252862
2                   Sales  6959.172646


# 3. Data Cleaning & Wrangling

* Check missing values

* Simulate & handle nulls

* Remove duplicates

* Handle outliers

* Encode categorical features

* Feature engineering

In [ ]:
# Check missing values
print("Missing values before cleaning:")
print(df.isnull().sum())

# Simulate missing values
df.loc[df.sample(frac=0.02).index, "NumCompaniesWorked"] = np.nan

# Check missing values after simulating and before filling
if df["NumCompaniesWorked"].isnull().sum() > 0:
    print("\nMissing values after simulating and before filling 'NumCompaniesWorked':")
    print(df.isnull().sum())

# Handle simulated nulls
df["NumCompaniesWorked"] = df["NumCompaniesWorked"].fillna(df["NumCompaniesWorked"].median())

# Check missing values after filling
print("\nMissing values after filling 'NumCompaniesWorked':")
print(df.isnull().sum())

# Check duplicates
print("\nNumber of duplicate rows before dropping:", df.duplicated().sum())
df.drop_duplicates(inplace=True)
print("Number of duplicate rows after dropping:", df.duplicated().sum())

# Encode categorical variables
le = LabelEncoder()
df["OverTime"] = le.fit_transform(df["OverTime"])

Missing values before cleaning:
Age                         0
Attrition                   0
BusinessTravel              0
DailyRate                   0
Department                  0
DistanceFromHome            0
Education                   0
EducationField              0
EmployeeCount               0
EmployeeNumber              0
EnvironmentSatisfaction     0
Gender                      0
HourlyRate                  0
JobInvolvement              0
JobLevel                    0
JobRole                     0
JobSatisfaction             0
MaritalStatus               0
MonthlyIncome               0
MonthlyRate                 0
NumCompaniesWorked          0
Over18                      0
OverTime                    0
PercentSalaryHike           0
PerformanceRating           0
RelationshipSatisfaction    0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears           0
TrainingTimesLastYear       0
WorkLifeBalance             0
YearsAtCompany              0
YearsInC